# SD - SR 

The following notebook allows to replicate the simulation as describe in Alamia et al. (2019). CNN and a Siamese networks are compared when performing identity (SD) and Spatial Relationship (SR) tasks. See Alamia et al. (2019) for further details.

After having imported a bunch of packages, we load the data from a previously genereated Matlab file.

In [3]:
%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt

import time
import csv
from keras.layers.core import Dense, Activation, Dropout, Flatten
from keras.layers.convolutional import Conv2D
from keras.layers import MaxPooling2D, Conv2D, ZeroPadding2D, Activation, Input, concatenate, Add, Multiply

from keras import backend as K
from keras.models import Sequential, load_model, Model
from keras import regularizers, optimizers
from keras.utils import np_utils
np.random.seed(1234)

import tensorflow as tf 
import keras
from keras.layers.merge import _Merge

import scipy.io as sio
from scipy import stats


Using TensorFlow backend.


In [4]:
def getImagesNlabelsSiam() :

    infile = sio.loadmat('siamese1000val.mat')
    
    allTrialsTr = infile.get('allTrialsTr')
    SRconditionsTr=infile.get('SRconditionsTr')
    SDconditionsTr=infile.get('SDconditionsTr')
    
    allTrialsVal = infile.get('allTrialsVal')
    SRconditionsVal=infile.get('SRconditionsVal')
    SDconditionsVal=infile.get('SDconditionsVal')
    
    allTrialsTe = infile.get('allTrialsTe')
    SRconditionsTe=infile.get('SRconditionsTe')
    SDconditionsTe=infile.get('SDconditionsTe')

    #shuffling training
    perm = np.arange(allTrialsTr.shape[3])  
    np.random.shuffle(perm)  
    allTrialsTrPerm = allTrialsTr[:,:,:,perm] 
    SRconditionsTrPerm = SRconditionsTr[:,perm] 
    SDconditionsTrPerm = SDconditionsTr[:,perm] 
   
    #providing it with the right dimensions  training
    allTrialsTrRearranged=np.moveaxis(allTrialsTrPerm,-1,0)
    SDconditionsTrRearranged=np.moveaxis(SDconditionsTrPerm,-1,0)
    SRconditionsTrRearranged=np.moveaxis(SRconditionsTrPerm,-1,0)
    allTrialsTrRearranged2 = np.expand_dims(allTrialsTrRearranged,axis=-1)
    allTrialsTrRearranged2 = allTrialsTrRearranged
    
    
    #shuffling validation
    perm = np.arange(allTrialsVal.shape[3])  
    np.random.shuffle(perm)  
    allTrialsValPerm = allTrialsVal[:,:,:,perm] 
    SRconditionsValPerm = SRconditionsVal[:,perm] 
    SDconditionsValPerm = SDconditionsVal[:,perm] 
   
    #providing it with the right dimensions validation
    allTrialsValRearranged=np.moveaxis(allTrialsValPerm,-1,0)
    SDconditionsValRearranged=np.moveaxis(SDconditionsValPerm,-1,0)
    SRconditionsValRearranged=np.moveaxis(SRconditionsValPerm,-1,0)
    allTrialsValRearranged2 = np.expand_dims(allTrialsValRearranged,axis=-1)
    allTrialsValRearranged2 = allTrialsValRearranged
    
    
    #shuffling testing
    perm = np.arange(allTrialsTe.shape[3])  
    np.random.shuffle(perm)  
    allTrialsTePerm = allTrialsTe[:,:,:,perm] 
    SRconditionsTePerm = SRconditionsTe[:,perm] 
    SDconditionsTePerm = SDconditionsTe[:,perm] 
   
    #providing it with the right dimensions  testing
    allTrialsTeRearranged=np.moveaxis(allTrialsTePerm,-1,0)
    SDconditionsTeRearranged=np.moveaxis(SDconditionsTePerm,-1,0)
    SRconditionsTeRearranged=np.moveaxis(SRconditionsTePerm,-1,0)
    allTrialsTeRearranged2 = np.expand_dims(allTrialsTeRearranged,axis=-1)
    allTrialsTeRearranged2 = allTrialsTeRearranged
    
    return {'imagesTr':allTrialsTrRearranged2,'SRlabelsTr':SRconditionsTrRearranged,'SDlabelsTr':SDconditionsTrRearranged,'imagesVal':allTrialsValRearranged2,'SRlabelsVal':SRconditionsValRearranged,'SDlabelsVal':SDconditionsValRearranged,'imagesTe':allTrialsTeRearranged2,'SRlabelsTe':SRconditionsTeRearranged,'SDlabelsTe':SDconditionsTeRearranged}

In [5]:
def getImagesNlabels() :

    #infile = sio.loadmat('artVisDataset.mat')
    infile = sio.loadmat('cnn1000val.mat')

    allTrialsTr = infile.get('allTrialsTr')
    SRconditionsTr=infile.get('SRconditionsTr')
    SDconditionsTr=infile.get('SDconditionsTr')
    
    allTrialsVal = infile.get('allTrialsVal')
    SRconditionsVal=infile.get('SRconditionsVal')
    SDconditionsVal=infile.get('SDconditionsVal')
    
    allTrialsTe = infile.get('allTrialsTe')
    SRconditionsTe=infile.get('SRconditionsTe')
    SDconditionsTe=infile.get('SDconditionsTe')
    

    #shuffling training
    perm = np.arange(allTrialsTr.shape[2])  
    np.random.shuffle(perm)  
    allTrialsTrPerm = allTrialsTr[:,:,perm] 
    SRconditionsTrPerm = SRconditionsTr[:,perm] 
    SDconditionsTrPerm = SDconditionsTr[:,perm] 
   
    #providing it with the right dimensions  training
    allTrialsTrRearranged=np.moveaxis(allTrialsTrPerm,-1,0)
    SDconditionsTrRearranged=np.moveaxis(SDconditionsTrPerm,-1,0)
    SRconditionsTrRearranged=np.moveaxis(SRconditionsTrPerm,-1,0)
    allTrialsTrRearranged2 = np.expand_dims(allTrialsTrRearranged,axis=-1)
    
        
    #shuffling validation
    perm = np.arange(allTrialsVal.shape[2])  
    np.random.shuffle(perm)  
    allTrialsValPerm = allTrialsVal[:,:,perm] 
    SRconditionsValPerm = SRconditionsVal[:,perm] 
    SDconditionsValPerm = SDconditionsVal[:,perm] 
   
    #providing it with the right dimensions validation
    allTrialsValRearranged=np.moveaxis(allTrialsValPerm,-1,0)
    SDconditionsValRearranged=np.moveaxis(SDconditionsValPerm,-1,0)
    SRconditionsValRearranged=np.moveaxis(SRconditionsValPerm,-1,0)
    allTrialsValRearranged2 = np.expand_dims(allTrialsValRearranged,axis=-1)
    allTrialsValRearranged2 = allTrialsValRearranged
    
    
    #shuffling testing
    perm = np.arange(allTrialsTe.shape[2])  
    np.random.shuffle(perm)  
    allTrialsTePerm = allTrialsTe[:,:,perm] 
    SRconditionsTePerm = SRconditionsTe[:,perm] 
    SDconditionsTePerm = SDconditionsTe[:,perm] 
   
    #providing it with the right dimensions  testing
    allTrialsTeRearranged=np.moveaxis(allTrialsTePerm,-1,0)
    SDconditionsTeRearranged=np.moveaxis(SDconditionsTePerm,-1,0)
    SRconditionsTeRearranged=np.moveaxis(SRconditionsTePerm,-1,0)
    allTrialsTeRearranged2 = np.expand_dims(allTrialsTeRearranged,axis=-1)
    
    return {'imagesTr':allTrialsTrRearranged2,'SRlabelsTr':SRconditionsTrRearranged,'SDlabelsTr':SDconditionsTrRearranged,'imagesTe':allTrialsTeRearranged2,'SRlabelsTe':SRconditionsTeRearranged,'SDlabelsTe':SDconditionsTeRearranged}

In the next two cells we define the Siamese Network and the convolutional one. Note that in the Siamese network we redefine the class Substract.

In [6]:
class Subtract(_Merge):

    def build(self, input_shape):
        super(Subtract, self).build(input_shape)
        if len(input_shape) != 2:
            raise ValueError('A `Subtract` layer should be called '
                             'on exactly 2 inputs')

    def _merge_function(self, inputs):
        if len(inputs) != 2:
            raise ValueError('A `Subtract` layer should be called '
                             'on exactly 2 inputs')
        return inputs[0] - inputs[1]


def get_siamese_model(images):

    # Define the tensors for the two input images
#     left_input = Input(images[:,:,:,0].shape)
#     right_input = Input(images[:,:,:,1].shape)
#     left_input = np.expand_dims(left_input,axis=-1)
#     right_input = np.expand_dims(right_input,axis=-1)
#     left_input = Input(left_input.shape)
#     right_input = Input(right_input.shape)
    
    input_shape = (images.shape[1], images.shape[2], 1)
    left_input = Input(input_shape)
    right_input = Input(input_shape)
    
    
    # Convolutional Neural Network
    model = Sequential()

    model.add(Conv2D(4, (2, 2), strides=1, input_shape=(images.shape[1], images.shape[2], 1), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=1))
    model.add(Dropout(0.3))
    
    model.add(Conv2D(4, (2, 2), strides=1, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=1))
    model.add(Dropout(0.3))

    model.add(Conv2D(4, (2, 2), strides=1, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=1))   
    model.add(Dropout(0.3))

    model.add(Conv2D(4, (2, 2), strides=1, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=1))
    model.add(Dropout(0.3))
    
    model.add(Conv2D(4, (2, 2), strides=1, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=1))
    model.add(Dropout(0.3))
    
    model.add(Conv2D(4, (2, 2), strides=1, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=1))
    model.add(Dropout(0.3))
    
    model.add(Flatten())
    
    # Generate the encodings (feature vectors) for the two images
    encoded_l = model(left_input)
    encoded_r = model(right_input)
    
    combineFeatures2 = Subtract()([encoded_l, encoded_r])
    
    # Add a dense layer with a sigmoid unit to generate the similarity score
    prediction1 = Dense(128,activation='relu')(combineFeatures2)
    prediction1do = Dropout(.3)(prediction1)
    prediction2 = Dense(128,activation='relu')(prediction1)
    prediction2do = Dropout(.3)(prediction2)
    prediction = Dense(1,activation='sigmoid')(prediction2do)
    
    # Connect the inputs with the outputs
    siamese_net = Model(inputs=[left_input,right_input],outputs=prediction)
    
    siamese_net.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer=optimizers.Adam(lr=0.0001))
    
    # return the model
    return siamese_net

In [7]:
def buildingTheNetwork(images) :
    
    model = Sequential()

    model.add(Conv2D(4, (2, 2), strides=1, input_shape=(images.shape[1], images.shape[2], 1), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=1))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(4, (2, 2), strides=1, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=1))
    model.add(Dropout(0.2))

    model.add(Conv2D(4, (2, 2), strides=1, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=1))   
    model.add(Dropout(0.2))

    model.add(Conv2D(4, (2, 2), strides=1, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=1))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(4, (2, 2), strides=1, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=1))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(4, (2, 2), strides=1, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=1))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(4, (2, 2), strides=1, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=1))
    model.add(Dropout(0.2))

    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(128, activation='relu'))

    model.add(Dense(1))
    model.add(Activation('sigmoid'))

    model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer=optimizers.Adam(lr=0.0001),)

    return model

Here we finally run the code

In [8]:
K.clear_session()

data=getImagesNlabels() 
imagesTr=data['imagesTr']
SRlabelsTr=data['SRlabelsTr']
SDlabelsTr=data['SDlabelsTr']

imagesTe=data['imagesTe']
SRlabelsTe=data['SRlabelsTe']
SDlabelsTe=data['SDlabelsTe']


dataSiam=getImagesNlabelsSiam() 
imagesTrSiam=dataSiam['imagesTr']
SRlabelsTrSiam=dataSiam['SRlabelsTr']
SDlabelsTrSiam=dataSiam['SDlabelsTr']

imagesValSiam=dataSiam['imagesVal']
SRlabelsValSiam=dataSiam['SRlabelsVal']
SDlabelsValSiam=dataSiam['SDlabelsVal']

imagesTeSiam=dataSiam['imagesTe']
SRlabelsTeSiam=dataSiam['SRlabelsTe']
SDlabelsTeSiam=dataSiam['SDlabelsTe']


imagesTrNewL=imagesTrSiam[:,:,:,0]
imagesTrNewR=imagesTrSiam[:,:,:,1]
imagesTrNewL = np.expand_dims(imagesTrNewL,axis=-1)
imagesTrNewR = np.expand_dims(imagesTrNewR,axis=-1)

imagesValNewL=imagesValSiam[:,:,:,0]
imagesValNewR=imagesValSiam[:,:,:,1]
imagesValNewL = np.expand_dims(imagesValNewL,axis=-1)
imagesValNewR = np.expand_dims(imagesValNewR,axis=-1)

imagesTeNewL=imagesTeSiam[:,:,:,0]
imagesTeNewR=imagesTeSiam[:,:,:,1]
imagesTeNewL = np.expand_dims(imagesTeNewL,axis=-1)
imagesTeNewR = np.expand_dims(imagesTeNewR,axis=-1)

networkSR=buildingTheNetwork(imagesTr) 
networkSD=buildingTheNetwork(imagesTr) 

siamNetSD=get_siamese_model(imagesTrSiam) 

networkSD.summary()
networkSR.summary()
siamNetSD.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 49, 49, 4)         20        
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 48, 48, 4)         0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 48, 48, 4)         0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 47, 47, 4)         68        
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 46, 46, 4)         0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 46, 46, 4)         0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 45, 45, 4)         68        
__________

and finally we fit the models!

In [9]:
historySR = networkSR.fit(
            imagesTr, SRlabelsTr,
            batch_size=50, epochs=70, shuffle=False) 

historySD = networkSD.fit(
            imagesTr, SDlabelsTr,
            batch_size=50, epochs=70, shuffle=False) 

historySiam = siamNetSD.fit(
            [imagesTrNewR, imagesTrNewL], [SDlabelsTrSiam],
            batch_size=50, epochs=70, validation_split=0.0, shuffle=False) 


Epoch 1/70
1000/1000 [==============================] - 4s - loss: 0.6621 - acc: 0.7290     
Epoch 2/70
1000/1000 [==============================] - 3s - loss: 0.5489 - acc: 0.9170     
Epoch 3/70
1000/1000 [==============================] - 3s - loss: 0.3657 - acc: 0.9310     
Epoch 4/70
1000/1000 [==============================] - 3s - loss: 0.2226 - acc: 0.9410     
Epoch 5/70
1000/1000 [==============================] - 4s - loss: 0.1491 - acc: 0.9510     
Epoch 6/70
1000/1000 [==============================] - 4s - loss: 0.1164 - acc: 0.9550     
Epoch 7/70
1000/1000 [==============================] - 4s - loss: 0.0770 - acc: 0.9780     
Epoch 8/70
1000/1000 [==============================] - 4s - loss: 0.0589 - acc: 0.9870     
Epoch 9/70
1000/1000 [==============================] - 4s - loss: 0.0576 - acc: 0.9830     
Epoch 10/70
1000/1000 [==============================] - 4s - loss: 0.0361 - acc: 0.9940     
Epoch 11/70
1000/1000 [==============================] - 4s - loss: 0

1000/1000 [==============================] - 3s - loss: 0.4467 - acc: 0.7950     
Epoch 17/70
1000/1000 [==============================] - 3s - loss: 0.4033 - acc: 0.8330     
Epoch 18/70
1000/1000 [==============================] - 3s - loss: 0.3948 - acc: 0.8250     
Epoch 19/70
1000/1000 [==============================] - 3s - loss: 0.3708 - acc: 0.8450     
Epoch 20/70
1000/1000 [==============================] - 3s - loss: 0.3434 - acc: 0.8540     
Epoch 21/70
1000/1000 [==============================] - 3s - loss: 0.2931 - acc: 0.8890     
Epoch 22/70
1000/1000 [==============================] - 3s - loss: 0.2830 - acc: 0.8740     
Epoch 23/70
1000/1000 [==============================] - 3s - loss: 0.2431 - acc: 0.9080     
Epoch 24/70
1000/1000 [==============================] - 3s - loss: 0.2460 - acc: 0.8950     
Epoch 25/70
1000/1000 [==============================] - 3s - loss: 0.1991 - acc: 0.9180     
Epoch 26/70
1000/1000 [==============================] - 3s - loss: 0.21

1000/1000 [==============================] - 7s - loss: 0.4617 - acc: 0.7730 - val_loss: 0.6322 - val_acc: 0.6270
Epoch 25/70
1000/1000 [==============================] - 7s - loss: 0.4524 - acc: 0.7890 - val_loss: 0.6232 - val_acc: 0.6500
Epoch 26/70
1000/1000 [==============================] - 7s - loss: 0.4130 - acc: 0.7970 - val_loss: 0.6121 - val_acc: 0.6840
Epoch 27/70
1000/1000 [==============================] - 7s - loss: 0.4091 - acc: 0.8070 - val_loss: 0.6047 - val_acc: 0.6790
Epoch 28/70
1000/1000 [==============================] - 7s - loss: 0.3799 - acc: 0.8300 - val_loss: 0.5864 - val_acc: 0.7670
Epoch 29/70
1000/1000 [==============================] - 7s - loss: 0.3692 - acc: 0.8350 - val_loss: 0.5905 - val_acc: 0.7180
Epoch 30/70
1000/1000 [==============================] - 7s - loss: 0.3110 - acc: 0.8820 - val_loss: 0.5807 - val_acc: 0.7050
Epoch 31/70
1000/1000 [==============================] - 7s - loss: 0.3561 - acc: 0.8360 - val_loss: 0.5655 - val_acc: 0.7450
Epoc

In [10]:
networkSR.save("networkSR.h5")
networkSD.save("siamNetSD.h5")
siamNetSD.save("siamNetSD.h5")

# networkSR = load_model('networkSR.h5')

In [13]:
perfSR=[]
perfSD=[]
predSD = networkSD.predict(imagesTe)
predSD=np.around(predSD)
resTeSD=np.subtract(SDlabelsTe.flatten(),predSD.flatten())
perfSD.append(1-sum(abs(resTeSD))/len(resTeSD))

predSR = networkSR.predict(imagesTe)
predSR=np.around(predSR)
resTeSR=np.subtract(SRlabelsTe.flatten(),predSR.flatten())
perfSR.append(1-sum(abs(resTeSR))/len(resTeSR))

print(np.mean(perfSD))
print(np.mean(perfSR))

0.5
0.999


In [14]:
perfSDsia=[]
predSDsia = siamNetSD.predict([imagesTeNewR,imagesTeNewL])
predSDsia=np.around(predSDsia)
resTeSDsia=np.subtract(SDlabelsTeSiam.flatten(),predSDsia.flatten())
perfSDsia.append(1-sum(abs(resTeSDsia))/len(resTeSDsia))
print(np.mean(perfSDsia))


0.958


Next we save all the results for further analysis in Matlab.

In [11]:
sio.savemat('perfSR01.mat', mdict={'perfSR':perfSR})
sio.savemat('perfSD01.mat', mdict={'perfSD':perfSD})

historyAccSR=historySR.history['acc']
historyAccSD=historySD.history['acc']

sio.savemat('historyAccSR01.mat', mdict={'historyAccSR':historyAccSR})
sio.savemat('historyAccSD01.mat', mdict={'historyAccSD':historyAccSD})


sio.savemat('perfSD10sia.mat', mdict={'perfSDsia':perfSDsia})

historyAccSDsia=historySiam.history['acc']

sio.savemat('historyAccSDsia10.mat', mdict={'historyAccSDsia':historyAccSDsia})
